In [172]:
import sqlite3
import random
import numpy as np
from faker import Faker
import pandas as pd
from datetime import date, timedelta

# Inicializar o Faker para dados em Português
faker = Faker('pt_PT')

In [173]:
Departamentos = ["Recursos Humanos", "Tecnologia da Informação", "Financeiro", "Marketing", "Vendas"]
Cargos = ["Analista", "Desenvolvedor", "Gerente", "Coordenador", "Assistente"]

In [174]:
def criar_departamentos():
    lista_departamentos = []
    for i in range(1, len(Departamentos) + 1):
        lista_departamentos.append({
            "ID_depart": i,
            "Nome": Departamentos[i-1],
            "ID_gerente": None # Começa como Nulo
        })
    return pd.DataFrame(lista_departamentos)



In [175]:
def criar_funcionarios(n=20):
    lista_funcionarios = []
    for i in range(1, n + 1):
        lista_funcionarios.append({
            "ID_fun": i,
            "NIF": str(283684274 + i),
            "Primeiro_nome": faker.first_name(),
            "Ultimo_nome": faker.last_name(),
            "Nome_rua": faker.street_name(),
            "Nome_localidade": faker.city(),
            "Codigo_postal": faker.postcode(),
            "Num_telemovel": faker.phone_number(),
            "Email": faker.unique.email(),
            "Data_nascimento": faker.date_of_birth(minimum_age=18, maximum_age=65),
            "Cargo": random.choice([cargo for cargo in Cargos if cargo != "Gerente"]),
            "ID_depart": np.random.randint(1, len(Departamentos) + 1)
        })
    return pd.DataFrame(lista_funcionarios)

In [176]:
Departamentos = criar_departamentos()
Funcionarios = criar_funcionarios()
Departamentos["ID_gerente"] = [np.random.randint(1, len(Funcionarios) + 1) for i in range(len(Departamentos))]



In [177]:
Departamentos


,ID_depart,Nome,ID_gerente
0,1,Recursos Humanos,8
1,2,Tecnologia da Informação,10
2,3,Financeiro,14
3,4,Marketing,6
4,5,Vendas,12


In [179]:
Funcionarios.head()

,ID_fun,NIF,Primeiro_nome,Ultimo_nome,Nome_rua,Nome_localidade,Codigo_postal,Num_telemovel,Email,Data_nascimento,Cargo,ID_depart
0,1,283684275,Fernando,Pinheiro,Rua Ribeirinhos,Portalegre,4785-813,932651728,edgar85@example.org,1984-07-07,Analista,4
1,2,283684276,Lúcia,Figueiredo,Avenida Vasco Jesus,Montemor-o-Novo,7835-222,289 019 639,tatiana23@example.com,1993-05-09,Assistente,1
2,3,283684277,Rita,Lopes,Rua das Cebolas,Queluz,1079-315,(351) 921965435,pedro62@example.com,1996-06-02,Analista,5
3,4,283684278,Gustavo,Figueiredo,Av Andrade,Guarda,4964-705,929823423,camposdiana@example.org,1980-04-15,Assistente,4
4,5,283684279,Iara,Araújo,Avenida de Cunha,Ponte de Sor,6461-189,(351) 914817037,beneditajesus@example.com,1989-03-04,Analista,1


In [ ]:
# Adicionar os departamentos à tabela sem os ids dos gerentes
# conn = sqlite3.connect("C:/Users/diogo/OneDrive/Documentos/FCUL/Bases de dados/projeto_bd/Tabelas_teste.db")
# # departamentos_sem_gerente_df['ID_gerente'] = None
# departamentos_sem_gerente_df.to_sql('Departamentos', conn, if_exists='append', index=False)
# funcionarios_df.to_sql('Funcionarios', conn, if_exists='append', index=False)
# cursor = conn.cursor()
#for index, row in departamentos_df.iterrows():
  #  if pd.notna(row['ID_gerente']):
   #     cursor.execute("UPDATE Departamentos SET ID_gerente = ? WHERE ID_depart = ?", (int(row['ID_gerente']), int(row['ID_depart'])))
#conn.commit()
#conn.close()

In [180]:
def criar_remuneracoes_salarios_beneficios(funcionarios_df):
    """
    Gera dados para as tabelas Remuneracoes, Salario e Beneficios.
    A Data_fim tem 50% de chance de ser nula (contrato ativo) e 50% de chance
    de ser uma data futura.
    """
    lista_remuneracoes = []
    lista_salarios = []
    lista_beneficios = []

    tipos_de_beneficios = ['Subsídio Alimentação', 'Seguro Saúde', 'Carro Empresa', 'Subsídio Transporte', 'Telemóvel Empresa']

    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # 1. Criar o registo na tabela Remuneracoes
        data_inicio = faker.date_between(start_date='-5y', end_date='today')
        
        # 50% de chance de o contrato ter terminado, 50% de estar ativo (None)
        # Se terminou, a data de fim é entre 3 meses e 4 anos depois do início.
        data_fim = (data_inicio + timedelta(days=random.randint(90, 1460))) if random.random() < 0.5 else None

        remuneracao_record = {
            "ID_fun": id_fun,
            "Data_inicio": data_inicio,
            "Data_fim": data_fim
        }
        lista_remuneracoes.append(remuneracao_record)
        
        # 2. Criar o registo correspondente na tabela Salario
        salario_bruto = round(np.random.normal(loc=1600, scale=700), 2)
        if salario_bruto < 950: salario_bruto = 950 # Salário mínimo aprox. para 2025

        salario_record = {
            "ID_fun": id_fun,
            "Data_inicio": data_inicio,
            "salario_bruto": salario_bruto,
        }
        lista_salarios.append(salario_record)
        
        # 3. Criar zero ou mais registos na tabela Beneficios, a chance de ter beneficios é 80%
        if random.random() < 0.8:
            num_beneficios = random.randint(1, 3)
            beneficios_para_este_fun = random.sample(tipos_de_beneficios, num_beneficios)
            
            for tipo_beneficio in beneficios_para_este_fun:
                beneficio_record = {
                    "ID_fun": id_fun,
                    "Data_inicio": data_inicio,
                    "tipo": tipo_beneficio,
                    "valor": round(random.uniform(30, 450), 2),
                }
                lista_beneficios.append(beneficio_record)

    return pd.DataFrame(lista_remuneracoes), pd.DataFrame(lista_salarios), pd.DataFrame(lista_beneficios)

In [181]:
Remuneracoes, Salarios, Beneficios = criar_remuneracoes_salarios_beneficios(Funcionarios)

In [182]:
Remuneracoes

,ID_fun,Data_inicio,Data_fim
0,1,2021-01-23,2022-12-13
1,2,2023-09-30,None
2,3,2021-01-23,None
3,4,2022-05-01,None
4,5,2022-04-02,2024-07-29
5,6,2023-09-08,2025-10-30
6,7,2020-11-19,None
7,8,2025-09-23,None
8,9,2021-10-23,None
9,10,2021-11-02,None


In [183]:
Salarios

,ID_fun,Data_inicio,salario_bruto
0,1,2021-01-23,1697.29
1,2,2023-09-30,1418.00
2,3,2021-01-23,1285.35
3,4,2022-05-01,1544.84
4,5,2022-04-02,1831.52
5,6,2023-09-08,1526.69
6,7,2020-11-19,2195.75
7,8,2025-09-23,2187.38
8,9,2021-10-23,2045.45
9,10,2021-11-02,1725.99


In [184]:
Beneficios

,ID_fun,Data_inicio,tipo,valor
0,1,2021-01-23,Seguro Saúde,321.20
1,3,2021-01-23,Subsídio Transporte,348.80
2,3,2021-01-23,Subsídio Alimentação,90.82
3,3,2021-01-23,Seguro Saúde,255.87
4,4,2022-05-01,Telemóvel Empresa,55.67
5,4,2022-05-01,Subsídio Alimentação,52.01
6,5,2022-04-02,Seguro Saúde,448.20
7,5,2022-04-02,Subsídio Alimentação,411.46
8,5,2022-04-02,Telemóvel Empresa,410.03
9,7,2020-11-19,Telemóvel Empresa,323.16


In [185]:
def criar_ferias(funcionarios_df):
    """
    NÃO TEM O NUM_DIAS_PERMITIDOS
    Gera um histórico de férias para os funcionários.
    Cada funcionário pode ter entre 0 a 4 registos de férias.
    """
    lista_ferias = []
    estados_possiveis = ['Aprovado', 'Aprovado', 'Aprovado', 'Aprovado', 'Rejeitado', 'Por aprovar']
    
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        num_periodos_ferias = random.randint(0, 4)
        
        for _ in range(num_periodos_ferias):
            data_inicio = faker.date_between(start_date='-3y', end_date='today')
            num_dias = random.randint(2, 15)
            data_fim = data_inicio + timedelta(days=num_dias)
            
            ferias_record = {
                "ID_fun": id_fun,
                "data_inicio": data_inicio,
                "data_fim": data_fim,
                "num_dias": num_dias,
                "estado_aprov": random.choice(estados_possiveis)
            }
            lista_ferias.append(ferias_record)
            
    return pd.DataFrame(lista_ferias)

In [186]:
Ferias = criar_ferias(Funcionarios)
Ferias

,ID_fun,data_inicio,data_fim,num_dias,estado_aprov
0,1,2023-03-20,2023-03-25,5,Aprovado
1,1,2023-03-25,2023-03-29,4,Aprovado
2,1,2024-04-11,2024-04-26,15,Aprovado
3,2,2023-09-30,2023-10-14,14,Aprovado
4,2,2024-11-10,2024-11-17,7,Por aprovar
5,2,2022-12-13,2022-12-15,2,Aprovado
6,3,2024-02-18,2024-02-29,11,Aprovado
7,3,2024-05-08,2024-05-19,11,Aprovado
8,3,2025-05-15,2025-05-30,15,Aprovado
9,3,2022-12-18,2022-12-20,2,Por aprovar


In [187]:
def criar_dependentes(funcionarios_df):
    """
    Gera uma lista de dependentes e depois limpa os dados para garantir que
    nenhum funcionário tem mais do que um dependente com o parentesco 'Pai/Mãe'.
    """
    lista_dependentes = []
    # Usamos 'Pai/Mãe' como uma única categoria para simplificar
    parentescos_possiveis = ['Filho(a)', 'Cônjuge', 'Enteado(a)', 'Pai/Mãe', 'Filho(a)'] # Repetimos 'Filho(a)' para aumentar a probabilidade
    sexos_possiveis = ['Masculino', 'Feminino', 'Outro']
    
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        if random.random() < 0.7:
            num_dependentes = random.randint(1, 4) # Aumentei para 4 para ter mais chance de criar duplicados para limpar
            
            for _ in range(num_dependentes):
                dependente_record = {
                    "ID_Fun": id_fun,
                    "nome": faker.name(),
                    "sexo": random.choice(sexos_possiveis),
                    "data_nascimento": faker.date_of_birth(minimum_age=0, maximum_age=80),
                    "parentesco": random.choice(parentescos_possiveis)
                }
                lista_dependentes.append(dependente_record)
                
    # --- PASSO DE LIMPEZA ---

    # 1. Criar o DataFrame com os dados
    if not lista_dependentes:
        return pd.DataFrame() # Retorna um DataFrame vazio se nenhum dependente foi gerado
        
    dependentes_bruto_df = pd.DataFrame(lista_dependentes)
    
    # 2. Separar os pais/mães dos outros dependentes
    pais_maes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] == 'Pai/Mãe']
    outros_dependentes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] != 'Pai/Mãe']
    
    # 3. Remover duplicados APENAS da parte dos pais/mães
    # drop_duplicates mantém a primeira ocorrência de cada combinação de 'ID_Fun' e 'parentesco'
    pais_maes_unicos_df = pais_maes_df.drop_duplicates(subset=['ID_Fun', 'parentesco'], keep='first')
    
    # 4. Juntar novamente as duas partes para ter o DataFrame final e limpo
    dependentes_final_df = pd.concat([pais_maes_unicos_df, outros_dependentes_df], ignore_index=True)
    
    return dependentes_final_df

In [189]:
Dependentes = criar_dependentes(Funcionarios)
Dependentes.head()

,ID_Fun,nome,sexo,data_nascimento,parentesco
0,1,Guilherme Leal,Outro,2025-01-26,Pai/Mãe
1,5,Kévim Pinho,Masculino,1969-07-02,Pai/Mãe
2,10,Carolina Oliveira,Outro,1959-03-10,Pai/Mãe
3,11,Constança Araújo-Carvalho,Feminino,2015-01-06,Pai/Mãe
4,16,Lorena de Anjos,Outro,1952-12-30,Pai/Mãe


In [190]:
def criar_faltas(funcionarios_df):
    """
    Gera um histórico de faltas para os funcionários.
    Cada funcionário terá entre 0 e 10 faltas registadas.
    """
    lista_faltas = []
    justificacoes_comuns = ["Consulta médica","Doença súbita","Apoio à família","Assuntos pessoais urgentes","Procedimentos legais/burocráticos","Avaria de transporte", None, None] # Para simular faltas injustificadas

    # Itera sobre cada funcionário para criar o seu histórico de faltas
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # Cada funcionário terá um número aleatório de faltas (de 0 a 10)
        num_faltas = int(np.random.normal(loc=3, scale=2))
        
        # Para evitar que as faltas do mesmo funcionário sejam todas no mesmo dia,
        # geramos um conjunto de datas únicas para este funcionário
        datas_possiveis = [faker.date_between(start_date='-3y', end_date='today') for _ in range(num_faltas)]
        datas_unicas = set(datas_possiveis) # 'set' remove datas duplicadas automaticamente
        
        for data_falta in datas_unicas:
            falta_record = {
                "ID_Fun": id_fun,
                "Data": data_falta,
                "Justificacao": random.choice(justificacoes_comuns)
            }
            lista_faltas.append(falta_record)
            
    return pd.DataFrame(lista_faltas)

In [192]:
Faltas = criar_faltas(Funcionarios)
Faltas.head()

,ID_Fun,Data,Justificacao
0,2,2024-12-20,Doença súbita
1,2,2024-04-30,Assuntos pessoais urgentes
2,2,2024-05-15,None
3,2,2024-03-22,Doença súbita
4,3,2023-02-15,Avaria de transporte


In [193]:
def criar_historico_empresas(funcionarios_df):
    """
    Gera um histórico profissional para os funcionários com nomes de departamento realistas.
    As datas são geradas de forma cronológica invertida para garantir consistência.
    """
    lista_historico = []

    # Lista de nomes de departamentos genéricos para empresas anteriores.
    departamentos_genericos = [
        'Marketing', 'Vendas', 'Tecnologia da Informação', 'Recursos Humanos',
        'Financeiro', 'Operações', 'Logística', 'Suporte ao Cliente',
        'Produto', 'Engenharia', 'Qualidade', 'Administrativo'
    ]
    
    # Itera sobre cada funcionário
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # 80% de chance de o funcionário ter um histórico profissional
        if random.random() < 0.8:
            num_historicos = random.randint(1, 3)
            
            latest_end_date = date.today() - timedelta(days=random.randint(90, 730))
            
            for i in range(num_historicos):
                data_fim = latest_end_date
                data_inicio = data_fim - timedelta(days=random.randint(365, 365 * 5))
                
                historico_record = {
                    "ID_Fun": id_fun,
                    "Nome_empresa": faker.company(),
                    # Agora escolhemos um departamento aleatório da lista
                    "Nome_departamento": random.choice(departamentos_genericos),
                    "Cargo": faker.job(),
                    "Data_inicio": data_inicio,
                    "Data_fim": data_fim
                }
                lista_historico.append(historico_record)
                
                latest_end_date = data_inicio - timedelta(days=random.randint(30, 365))

    return pd.DataFrame(lista_historico)

In [195]:
Historico_empresas = criar_historico_empresas(Funcionarios)
Historico_empresas.head()

,ID_Fun,Nome_empresa,Nome_departamento,Cargo,Data_inicio,Data_fim
0,1,Fonseca Lda.,Financeiro,Programador de aplicações,2023-12-16,2025-02-02
1,1,Amorim,Engenharia,Operador de instalações para o fabrico de vidro,2019-06-06,2022-12-22
2,2,Moreira Gomes S/A,Administrativo,Sericicultor e trabalhador qualificado da seri...,2022-01-27,2024-06-09
3,2,Freitas,Operações,Engenheiro eletrotécnico,2017-05-25,2021-06-16
4,2,Coelho,Qualidade,Assentador de tacos e afagador de madeira,2013-11-24,2016-12-07


In [196]:
def criar_modulo_recrutamento(funcionarios_df, departamentos_df, num_candidatos=50, num_vagas=30):
    """
    Gera dados para as tabelas Candidatos, Vagas, Requisitos_vaga e Candidato_a.
    """
    lista_candidatos = []
    lista_vagas = []
    lista_requisitos = []
    lista_candidaturas = []

    # --- 1. Gerar Candidatos ---
    for i in range(1, num_candidatos + 1):
        candidato_record = {
            "ID_cand": i,
            "Nome": faker.name(),
            "Email": faker.unique.email(),
            "Telemovel": faker.phone_number(),
            "CV": None, # BLOBs são representados como None
            "Carta_motivação": None
        }
        lista_candidatos.append(candidato_record)
    candidatos_df = pd.DataFrame(lista_candidatos)
    ids_candidatos_disponiveis = candidatos_df['ID_cand'].tolist()

    # --- 2. Gerar Vagas ---
    ids_departamentos_disponiveis = departamentos_df['ID_depart'].tolist()
    estados_vaga = ['Aberta', 'Fechada', 'Fechada', 'Fechada', 'Suspensa'] # Mais chance de estar fechada

    for i in range(1, num_vagas + 1):
        vaga_record = {
            "ID_vaga": i,
            "Data_abertura": faker.date_between(start_date='-2y', end_date='today'),
            "Estado": random.choice(estados_vaga),
            "ID_depart": random.choice(ids_departamentos_disponiveis)
        }
        lista_vagas.append(vaga_record)
    vagas_df = pd.DataFrame(lista_vagas)
    
    # --- 3. Gerar Requisitos para cada Vaga ---
    requisitos_possiveis = [
        'Python', 'SQL', 'Gestão de Projetos', 'Comunicação', 'Inglês Fluente',
        'Análise de Dados', 'React', 'Liderança', 'Trabalho em Equipa', 'Excel Avançado',
        'Marketing Digital', 'Contabilidade', 'Java', 'Atendimento ao Cliente'
    ]
    for index, vaga in vagas_df.iterrows():
        # Cada vaga terá entre 2 e 5 requisitos
        num_requisitos = random.randint(2, 5)
        # Usamos random.sample para garantir que os requisitos para a mesma vaga são únicos
        requisitos_para_vaga = random.sample(requisitos_possiveis, num_requisitos)
        
        for req in requisitos_para_vaga:
            requisito_record = {
                "ID_vaga": vaga['ID_vaga'],
                "Requisito": req
            }
            lista_requisitos.append(requisito_record)
    requisitos_df = pd.DataFrame(lista_requisitos)

    # --- 4. Gerar Candidaturas (Candidato_a) ---
    # Para ser mais realista, vamos pegar nos IDs dos funcionários de RH para serem os recrutadores
    id_depart_rh = departamentos_df[departamentos_df['Nome'] == 'Recursos Humanos']['ID_depart'].iloc[0]
    ids_recrutadores = funcionarios_df[funcionarios_df['ID_depart'] == id_depart_rh]['ID_fun'].tolist()
    # Fallback: se não houver ninguém em RH, usamos qualquer funcionário
    if not ids_recrutadores:
        ids_recrutadores = funcionarios_df['ID_fun'].tolist()
        
    estados_candidatura = ['Submetido', 'Em análise', 'Entrevista', 'Rejeitado', 'Rejeitado', 'Rejeitado', 'Contratado']

    # Cada candidato vai candidatar-se a um número aleatório de vagas (0 a 3)
    for id_candidato in ids_candidatos_disponiveis:
        num_candidaturas = random.randint(0, 3)
        if num_candidaturas > 0:
            # Escolhe 'num_candidaturas' vagas aleatórias e únicas para se candidatar
            vagas_aplicadas = vagas_df.sample(num_candidaturas)
            
            for index, vaga in vagas_aplicadas.iterrows():
                # A data da candidatura tem de ser posterior à data de abertura da vaga
                data_candidatura = faker.date_between(start_date=vaga['Data_abertura'], end_date='today')
                
                candidatura_record = {
                    "ID_cand": id_candidato,
                    "ID_Vaga": vaga['ID_vaga'],
                    "Data_cand": data_candidatura,
                    "Estado": random.choice(estados_candidatura),
                    "ID_recrutador": random.choice(ids_recrutadores)
                }
                lista_candidaturas.append(candidatura_record)
    candidato_a_df = pd.DataFrame(lista_candidaturas)
    
    return candidatos_df, vagas_df, requisitos_df, candidato_a_df

In [197]:
Candidatos, Vagas, Requisitos_vaga, Candidato_a = criar_modulo_recrutamento(Funcionarios, Departamentos)

In [198]:
Candidatos.head()

,ID_cand,Nome,Email,Telemovel,CV,Carta_motivação
0,1,Emília Castro,gabreu@example.org,(351) 911 445 290,None,None
1,2,Vítor Ferreira,oneves@example.org,964562020,None,None
2,3,Gustavo Araújo,qfaria@example.com,(351) 935 397 833,None,None
3,4,Sandro Pinto,camilagaspar@example.net,(351) 914 422 424,None,None
4,5,Eduarda Gaspar,pmoreira@example.org,(351) 923 635 010,None,None


In [199]:
Vagas.head()

,ID_vaga,Data_abertura,Estado,ID_depart
0,1,2025-03-24,Aberta,1
1,2,2025-05-19,Suspensa,2
2,3,2025-04-13,Fechada,4
3,4,2025-05-09,Fechada,1
4,5,2024-09-07,Fechada,1


In [200]:
Requisitos_vaga


,ID_vaga,Requisito
0,1,Java
1,1,Gestão de Projetos
2,1,Excel Avançado
3,1,Contabilidade
4,1,Análise de Dados
...,...,...
104,30,SQL
105,30,Comunicação
106,30,React
107,30,Inglês Fluente


In [201]:
Candidato_a

,ID_cand,ID_Vaga,Data_cand,Estado,ID_recrutador
0,2,3,2025-08-01,Rejeitado,2
1,2,8,2024-10-15,Entrevista,11
2,2,7,2025-08-12,Contratado,17
3,3,7,2025-07-05,Em análise,17
4,3,4,2025-05-15,Rejeitado,2
...,...,...,...,...,...
71,45,15,2024-09-30,Contratado,17
72,48,15,2024-12-31,Em análise,2
73,48,29,2025-05-14,Submetido,2
74,50,19,2025-06-06,Rejeitado,11


In [202]:
def criar_catalogo_formacoes():
    """
    Cria um catálogo fixo e manual de 5 formações com descrições realistas.
    """
    dados_formacoes = [
        {"ID_for": 1, "Nome_formacao": 'Liderança e Gestão de Equipas', "Descricao": 'Desenvolver competências de liderança, motivação e gestão de conflitos para chefes de equipa.', "Data_inicio": date(2023, 10, 5), "Data_fim": date(2023, 10, 10), "Estado": 'Concluida'},
        {"ID_for": 2, "Nome_formacao": 'Cibersegurança para Colaboradores', "Descricao": 'Sensibilizar para as melhores práticas de segurança digital, phishing e proteção de dados.', "Data_inicio": date(2024, 3, 15), "Data_fim": date(2024, 3, 16), "Estado": 'Concluida'},
        {"ID_for": 3, "Nome_formacao": 'Análise de Dados com Python', "Descricao": 'Introdução à análise de dados utilizando as bibliotecas Pandas e Matplotlib.', "Data_inicio": date(2024, 9, 2), "Data_fim": date(2024, 9, 13), "Estado": 'Em curso'},
        {"ID_for": 4, "Nome_formacao": 'Técnicas de Negociação e Vendas', "Descricao": 'Aperfeiçoar técnicas de comunicação, persuasão e fecho de negócios para a equipa comercial.', "Data_inicio": date(2025, 2, 10), "Data_fim": date(2025, 2, 14), "Estado": 'Planeada'},
        {"ID_for": 5, "Nome_formacao": 'Inteligência Emocional no Trabalho', "Descricao": 'Gerir emoções, melhorar o autoconhecimento e a empatia nas relações interpessoais no trabalho.', "Data_inicio": date(2025, 5, 19), "Data_fim": date(2025, 5, 20), "Estado": 'Planeada'}
    ]
    
    return pd.DataFrame(dados_formacoes)

In [203]:
Formacoes = criar_catalogo_formacoes()
Formacoes

,ID_for,Nome_formacao,Descricao,Data_inicio,Data_fim,Estado
0,1,Liderança e Gestão de Equipas,"Desenvolver competências de liderança, motivaç...",2023-10-05,2023-10-10,Concluida
1,2,Cibersegurança para Colaboradores,Sensibilizar para as melhores práticas de segu...,2024-03-15,2024-03-16,Concluida
2,3,Análise de Dados com Python,Introdução à análise de dados utilizando as bi...,2024-09-02,2024-09-13,Em curso
3,4,Técnicas de Negociação e Vendas,"Aperfeiçoar técnicas de comunicação, persuasão...",2025-02-10,2025-02-14,Planeada
4,5,Inteligência Emocional no Trabalho,"Gerir emoções, melhorar o autoconhecimento e a...",2025-05-19,2025-05-20,Planeada


In [204]:
def criar_teve_formacao(funcionarios_df, formacoes_df):
    """
    Gera as inscrições dos funcionários nas formações de forma aleatória e simples,
    sem verificar o estado ou as datas das formações.
    """
    lista_teve_formacao = []

    # Se não houver formações no catálogo, não podemos fazer nada.
    if formacoes_df.empty:
        return pd.DataFrame()

    # Itera sobre cada funcionário
    for index, funcionario in funcionarios_df.iterrows():
        # 60% de chance de o funcionário ter tido alguma formação
        if random.random() < 0.6:
            
            # Número de formações que este funcionário teve (1 ou 2)
            num_formacoes_feitas = random.randint(1, 2)
            
            # Escolhe aleatoriamente formações do catálogo COMPLETO
            formacoes_a_inscrever = formacoes_df.sample(num_formacoes_feitas)
            
            for i, formacao in formacoes_a_inscrever.iterrows():
                teve_formacao_record = {
                    "ID_Fun": funcionario['ID_fun'],
                    "ID_for": formacao['ID_for'],
                    "Certificado": None, # BLOB
                    "Data_inicio": formacao['Data_inicio'], # Copia as datas diretamente
                    "Data_fim": formacao['Data_fim']
                }
                lista_teve_formacao.append(teve_formacao_record)

    return pd.DataFrame(lista_teve_formacao)

In [205]:
Teve_Formacao = criar_teve_formacao(Funcionarios, Formacoes)
Teve_Formacao

,ID_Fun,ID_for,Certificado,Data_inicio,Data_fim
0,2,4,None,2025-02-10,2025-02-14
1,2,1,None,2023-10-05,2023-10-10
2,3,5,None,2025-05-19,2025-05-20
3,3,2,None,2024-03-15,2024-03-16
4,6,5,None,2025-05-19,2025-05-20
5,7,2,None,2024-03-15,2024-03-16
6,11,5,None,2025-05-19,2025-05-20
7,11,4,None,2025-02-10,2025-02-14
8,13,2,None,2024-03-15,2024-03-16
9,14,4,None,2025-02-10,2025-02-14
